In [1]:
# Import libraries 
import extract_eye_features
import dill
import scipy
import os
import numpy as np
import sys 

sys.path.append("/Users/zacharykelly/Documents/MATLAB/projects/lightLogger/raspberry_pi_firmware/utility")
import Pi_util
import matplotlib.pyplot as plt

Loading DLC 3.0.0rc8...
DLC loaded in light mode; you cannot use any GUI (labeling, relabeling and standalone GUI)


In [2]:
# Define the path to the folder containing the videos and other files 
path_to_video_folder: str = "/Users/zacharykelly/Desktop/geoff_videos_blnk"

# Generate paths to all of the videos in said folder 
video_paths: list[str] = [os.path.join(path_to_video_folder, filename)
                          for filename in os.listdir(path_to_video_folder)
                          if '.avi' in filename
                         ]

In [3]:
# Generate output folder 
output_folder_path: str = "/Users/zacharykelly/Aguirre-Brainard Lab Dropbox/Zachary Kelly/BLNK_analysis/PuffLight/blinkResponse/HERO_gka/videos"
if(not os.path.exists(output_folder_path)):
    os.mkdir(output_folder_path)

In [4]:
# Write the prediction function
def predict_eye_features(args: tuple) -> None:
    # Extract the arguments from the arguments tuple 
    filepath, crop_box, target_size, temp_dir_path, output_folder_path = args

    # Define the portion of the video to crop out 
    t, b, l, r = crop_box

    # Extract the name of this video
    video_name: str = os.path.splitext(os.path.basename(filepath.rstrip('/')))[0]

    # Find the FPS of the video 
    video_fps: float = Pi_util.inspect_video_FPS(filepath)

    # Videos are small, so we can load them entirely in from memory 
    video_as_arr: np.ndarray = Pi_util.destruct_video(filepath, is_grayscale=True)

    # Crop out only the eye from the video
    # and set the rest of the frame to black 
    video_cropped: np.ndarray = video_as_arr[:, t:b, l:r].copy()
    white_pixels: np.ndarray = video_cropped[0] > 200
    video_cropped[:, white_pixels] = 0

    # Resize the video to not a small resolution 
    y_offset = (target_size[0] - video_cropped.shape[1]) // 2
    x_offset = (target_size[1] - video_cropped.shape[2]) // 2

    video_resized: np.ndarray = np.zeros((len(video_cropped), *target_size), dtype=np.uint8)
    video_resized[:, y_offset:y_offset + video_cropped.shape[1], x_offset:x_offset + video_cropped.shape[2]] = video_cropped

    # Generate a temp video from this cropped video 
    temp_video_path: str = os.path.join(temp_dir_path, f"temp_{video_name}.avi")
    Pi_util.frames_to_video(video_resized, temp_video_path, video_fps)

    # Extract the eye features for this video
    eye_features: list[dict] = extract_eye_features.extract_eye_features(temp_video_path, is_grayscale=True, visualize_results=False, pupil_feature_method='pylids', safe_execution=True)

    # Save the features 
    scipy.io.savemat(os.path.join(output_folder_path, f"{video_name}_eye_features.mat"), 
                    {"eye_features": eye_features}
                    )
    
    # Remvove the temp avi video 
    os.remove(temp_video_path)


In [ ]:
# See how many CPU cores we have 
print(mp.cpu_count())

In [ ]:
# Generate a temp output directory 
temp_dir_path: str = './temp_blnk_pipeline'
if(not os.path.exists(temp_dir_path)):
    os.mkdir(temp_dir_path)

# Define the crop box and target size 
t, b, l, r = 140, 275, 190, 425
crop_box: tuple = (t, b, l, r)
target_size: tuple = (480, 640)

# Define the argument list that each function 
# will get 
args_list: list[list] = [ (filepath, crop_box, target_size, temp_dir_path, output_folder_path)
                          for filepath in video_paths 
                        ]

# Process the videos 
for video_num, video_args in enumerate(args_list):
    print(f"Processing: {video_num}/{len(args_list)}", flush=True)
    predict_eye_features(video_args)
    

Model weights already exist!
Analyzing videos with /Users/zacharykelly/Library/Application Support/pylids/pytorch-pupil-v1/dlc-models-pytorch/iteration-0/santini_eyelid_detectionJul182025-trainset99shuffle1/train/snapshot-best-220.pt
Starting to analyze temp_blnk_pipeline/temp_HERO_gka_blinkResponse_direction-LightFlux_contrast-0.05_block-2_sequence-2_trial-12_side-R.avi
Video metadata: 
  Overall # of frames:    450
  Duration of video [s]:  2.50
  fps:                    180.0
  resolution:             w=640, h=480

Running pose prediction with batch size 8


100%|██████████| 450/450 [00:14<00:00, 30.00it/s]


Saving results in /var/folders/g_/1p95771n5l1f_f8gbbftkjy80000gn/T/tmp2mq39387/temp_HERO_gka_blinkResponse_direction-LightFlux_contrast-0.05_block-2_sequence-2_trial-12_side-RDLC_Resnet50_santini_eyelid_detectionJul182025shuffle1_snapshot_220.h5 and /var/folders/g_/1p95771n5l1f_f8gbbftkjy80000gn/T/tmp2mq39387/temp_HERO_gka_blinkResponse_direction-LightFlux_contrast-0.05_block-2_sequence-2_trial-12_side-RDLC_Resnet50_santini_eyelid_detectionJul182025shuffle1_snapshot_220_full.pickle
The videos are analyzed. Now your research can truly start!
You can create labeled videos with 'create_labeled_video'.
If the tracking is not satisfactory for some videos, consider expanding the training set. You can use the function 'extract_outlier_frames' to extract a few representative outlier frames.

['/var/folders/g_/1p95771n5l1f_f8gbbftkjy80000gn/T/tmp2mq39387/temp_HERO_gka_blinkResponse_direction-LightFlux_contrast-0.05_block-2_sequence-2_trial-12_side-RDLC_Resnet50_santini_eyelid_detectionJul182025

100%|██████████| 450/450 [00:19<00:00, 22.69it/s]


Saving results in /var/folders/g_/1p95771n5l1f_f8gbbftkjy80000gn/T/tmpav7ieodb/temp_HERO_gka_blinkResponse_direction-LightFlux_contrast-0.05_block-2_sequence-2_trial-12_side-RDLC_Resnet50_santini_eyelid_detectionJul182025shuffle1_snapshot_110.h5 and /var/folders/g_/1p95771n5l1f_f8gbbftkjy80000gn/T/tmpav7ieodb/temp_HERO_gka_blinkResponse_direction-LightFlux_contrast-0.05_block-2_sequence-2_trial-12_side-RDLC_Resnet50_santini_eyelid_detectionJul182025shuffle1_snapshot_110_full.pickle
The videos are analyzed. Now your research can truly start!
You can create labeled videos with 'create_labeled_video'.
If the tracking is not satisfactory for some videos, consider expanding the training set. You can use the function 'extract_outlier_frames' to extract a few representative outlier frames.

['/var/folders/g_/1p95771n5l1f_f8gbbftkjy80000gn/T/tmpav7ieodb/temp_HERO_gka_blinkResponse_direction-LightFlux_contrast-0.05_block-2_sequence-2_trial-12_side-RDLC_Resnet50_santini_eyelid_detectionJul182025

...

Saving results in /var/folders/g_/1p95771n5l1f_f8gbbftkjy80000gn/T/tmpgzq6w8dy/temp_HERO_gka_blinkResponse_direction-LightFlux_contrast-0.05_block-1_sequence-2_trial-23_side-RDLC_Resnet50_santini_eyelid_detectionJul182025shuffle1_snapshot_220.h5 and /var/folders/g_/1p95771n5l1f_f8gbbftkjy80000gn/T/tmpgzq6w8dy/temp_HERO_gka_blinkResponse_direction-LightFlux_contrast-0.05_block-1_sequence-2_trial-23_side-RDLC_Resnet50_santini_eyelid_detectionJul182025shuffle1_snapshot_220_full.pickle
The videos are analyzed. Now your research can truly start!
You can create labeled videos with 'create_labeled_video'.
If the tracking is not satisfactory for some videos, consider expanding the training set. You can use the function 'extract_outlier_frames' to extract a few representative outlier frames.

['/var/folders/g_/1p95771n5l1f_f8gbbftkjy80000gn/T/tmpgzq6w8dy/temp_HERO_gka_blinkResponse_direction-LightFlux_contrast-0.05_block-1_sequence-2_trial-23_side-RDLC_Resnet50_santini_eyelid_detectionJul182025

100%|██████████| 450/450 [00:19<00:00, 22.73it/s]


Saving results in /var/folders/g_/1p95771n5l1f_f8gbbftkjy80000gn/T/tmp2sx8ghkr/temp_HERO_gka_blinkResponse_direction-LightFlux_contrast-0.05_block-1_sequence-2_trial-23_side-RDLC_Resnet50_santini_eyelid_detectionJul182025shuffle1_snapshot_110.h5 and /var/folders/g_/1p95771n5l1f_f8gbbftkjy80000gn/T/tmp2sx8ghkr/temp_HERO_gka_blinkResponse_direction-LightFlux_contrast-0.05_block-1_sequence-2_trial-23_side-RDLC_Resnet50_santini_eyelid_detectionJul182025shuffle1_snapshot_110_full.pickle
The videos are analyzed. Now your research can truly start!
You can create labeled videos with 'create_labeled_video'.
If the tracking is not satisfactory for some videos, consider expanding the training set. You can use the function 'extract_outlier_frames' to extract a few representative outlier frames.

['/var/folders/g_/1p95771n5l1f_f8gbbftkjy80000gn/T/tmp2sx8ghkr/temp_HERO_gka_blinkResponse_direction-LightFlux_contrast-0.05_block-1_sequence-2_trial-23_side-RDLC_Resnet50_santini_eyelid_detectionJul182025

 37%|███▋      | 167/450 [00:05<00:09, 29.81it/s]


KeyboardInterrupt: 